In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

##Install dependencies

In [ ]:
!pip install qiskit-aqua
!pip install qiskit

In [9]:
import qiskit
qiskit.__version__

'0.19.1'

##Importing dependencies

In [10]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance

from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

from qiskit import Aer

import math
import numpy as np
import pandas as pd
from sympy import *
import re
import time
import random
import itertools

## CSG problem solver Benchmark Dataset generator

###Different distributions data generator functions

Agent_based_uniform \\
Agent_based_normal \\
Beta_distribution \\
Modified_normal_distribution \\
Modified_uniform_distribution \\
Normal_distribution \\
SVA_BETA_distribution \\
Weibull_distribution \\
Rayleigh_distribution \\
Weighted_random_with_chisquare \\
F_distribution \\
Laplace_or_double_exponential \\

In [11]:
#Different distributions data generator functions

def Agent_based_uniform(y,Power,Numofagent):
  strx = bin(y)[2:]
  length = len(strx)
  Tempcoal = []
  for x in range(0, length):
      if (int(strx[x]) == 1):
          Tempcoal.append(length - x)
  sums = 0
  for j in Tempcoal :
      partial = np.random.uniform(0, 2 * Power.get(j))
      sums = sums + partial
  return sums


def Agent_based_normal(y,Power,Numofagent):
  strx = bin(y)[2:]
  length = len(strx)
  Tempcoal = []
  for x in range(0, length):
      if (int(strx[x]) == 1):
          Tempcoal.append(length - x)
  sums = 0
  for j in Tempcoal:
      partial = np.random.normal(Power.get(j), 0.01)
      sums = sums + partial
  return sums


def Beta_distribution(y,Power,Numofagent):
    return bin(y).count('1')* np.random.beta(.5, .5)


def Modified_normal_distribution(y,Power,Numofagent):
  mu = 10 * bin(y).count('1')
  sigma = 0.01
  value = np.random.normal(mu, sigma)
  randval = np.random.uniform(0, 50)
  if randval <= 20:
      value = value + randval
  return value


def Modified_uniform_distribution(y,Power,Numofagent):
  value= np.random.uniform(0,10*bin(y).count('1'))
  randval = np.random.uniform(0, 50)
  if randval <= 20:
      value = value + randval
  return  value


def Normal_distribution(y,Power,Numofagent):
  mu = 10 * bin(y).count('1')
  sigma = 0.01
  return np.random.normal(mu, sigma)


def SVA_BETA_distribution(y,Power,Numofagent):
  strx = bin(y)[2:]
  length = len(strx)
  Tempcoal = []
  for x in range(0, length):
      if (int(strx[x]) == 1):
          Tempcoal.append(length - x)
  x = len(Tempcoal)
  inputx = x * np.random.beta(0.5, 0.5)
  if (strx[len(strx) - 1] == str(1)):
      inputx = 200 + inputx
  return inputx


def Weibull_distribution(y,Power,Numofagent):
  lengthofcoal = bin(y).count('1')
  value = np.random.weibull(lengthofcoal * lengthofcoal)
  return value


def Rayleigh_distribution(y,Power,Numofagent):
  lengthofcoal = bin(y).count('1')
  mu = 10 * lengthofcoal
  modevalue = np.sqrt(2 / np.pi) * mu
  value = np.random.rayleigh(modevalue)
  return value


def Weighted_random_with_chisquare(y,Power,Numofagent):
  lengthofcoal = bin(y).count('1')
  value = random.randint(1, lengthofcoal)
  inputx = lengthofcoal * np.random.chisquare(lengthofcoal)
  totalvalue = value + inputx
  return totalvalue


def F_distribution(y,Power,Numofagent):
  lengthofcoal = bin(y).count('1')
  dfden = lengthofcoal + 1
  dfnum = 1
  value = np.random.f(dfnum, dfden)
  return value


def Laplace_or_double_exponential(y,Power,Numofagent):
  lengthofcoal = bin(y).count('1')
  value = np.random.laplace(10 * lengthofcoal, 0.1)
  return value


###Problem instance generator

In [12]:
def generate_problem_instance(distribution, NumAgent):
    totalcoalitions = 2 ** NumAgent
    y = 1
    Agents = list(range(1, NumAgent + 1))
    RealAgent = Agents[0:NumAgent]
    Power = {}
    for i in RealAgent:
        Power[i] = np.random.uniform(0, 10)
    coalition_values = {}
    while (y < totalcoalitions):
        sums = float("{0:.3f}".format(distribution(y,Power,NumAgent)))
        coalition_values[','.join([str(idx+1) for idx,bit in enumerate(bin(y)[2:][::-1]) if int(bit)])] = sums
        y = y + 1
    return coalition_values

##Function to convert CSG characteristic function to a BILP problem

In [13]:
def convert_to_BILP(coalition_values):
  """
  convert_to_BILP formulates the BILP problem for a given CSG problem instance
  :param coalition_values: dictionary of game/problem instance with key as the coalitions and value as coalition values
                          Also called the Characteristic function
  :return: tuple of (c,S,b) where c is the coalition values, S is a the binary 2D array and b is a binary vector
  """ 
  x={}
  for i in range(len(coalition_values)):
    x[i] = symbols(f'x_{i}')
  n = list(coalition_values.keys())[-1].count(',')+1                              #get number of agents
  S=[]
  for agent in range(n):
    temp = []
    for coalition in coalition_values.keys():
      if str(agent+1) in coalition:
        temp.append(1)                                                            #'1' if agent is present in coalition
      else:
        temp.append(0)                                                            #'0' if agent is not present in coalition
    S.append(temp)
  b=[1]*n                                                                         # vector b is a unit vector in case of BILP of CSGP
  c = list(coalition_values.values())                                             # vector of all costs (coalition values)
  return (c,S,b)

In [14]:
def get_QUBO_coeffs(c,S,b,P):
  """
  get_QUBO_coeffs converts the BILP problem instace into linear and quadratic terms required for QUBO formulation
  :param c: list of coalition values
         S: a binary 2D array
         b: a binary vector
         P: Penalty coefficient for the constraints of BILP problem to convert into an unconstrained QUBO problem
  :return linear: dictionary of linear coefficient terms
          quadratic: dictionary of quadratic coefficient terms
  """
  x={}
  for i in range(len(c)):
    x[i] = symbols(f'x_{i}')
  final_eq = simplify(sum([c_value*x[i] for i,c_value in enumerate(c)])+P*sum([expand((sum([x[idx] for idx,element in enumerate(agent) if element])-1)**2) for agent in S])) #simplify numerical equation
  linear={}
  quadratic={}
  for term in final_eq.as_coeff_add()[1]:
    term = str(term)
    if '**' in term:                                                              #get the coefficient of the squared terms as linear coefficients (diagonal elements of the Q matrix in the QUBO problem)
      if term.count('*')==3:
        linear[term.split('*')[1]] = float(term.split('*')[0])
      else:
        if not term.startswith('-'):
          linear[term.split('**')[0]] = float(1)
        else:
          linear[term.split('**')[0][1:]] = float(-1)
    elif term.count('*')==1:                                                        #get the coefficient of the linear terms (diagonal elements of the Q matrix in the QUBO problem)
      linear[term.split('*')[1]] += float(term.split('*')[0])
    else:
      quadratic[(term.split('*')[1],term.split('*')[2])] = float(term.split('*')[0])  #get the coefficient of quadratic terms (upper diagonal elements of the Q matrix of the QUBO problem)  
  linear = {k:-v for (k,v) in linear.items()}
  #{k: abs(v) for k, v in D.items()}
  quadratic = {k:-v for (k,v) in quadratic.items()}
  return linear,quadratic

###Utility functions

In [15]:
def natural_keys(text):
  """
  alist.sort(key=natural_keys) sorts in human order
  http://nedbatchelder.com/blog/200712/human_sorting.ht
  For example: Built-in function ['x_8','x_10','x_1'].sort() will sort as ['x_1', 'x_10', 'x_8']
  But using natural_keys as callback function for sort() will sort as ['x_1','x_8','x_10']
  """
  return [ atoi(c) for c in re.split(r'(\d+)', text) ]


def atoi(text):
  """
  Function returns the corresponding value of a numerical string as integer datatype
  """
  return int(text) if text.isdigit() else text

In [16]:
def qaoa_for_qubo(qubo, p=1):                   # QAOA solver for QUBO
  """
  qaoa_for_qubo solves the given QUBO using QAOA

  """
  aqua_globals.random_seed = 123
  initial_point = [np.random.uniform(2*np.pi) for _ in range(2*p)]
  quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),       #qasm simulator
                                     seed_simulator=aqua_globals.random_seed,
                                     seed_transpiler=aqua_globals.random_seed)
  qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=initial_point,p=p)
  qaoa = MinimumEigenOptimizer(qaoa_mes)    # using QAOA
  result = qaoa.solve(qubo)
  return result



def numpy_for_qubo(qubo, p=None):                      # Classical solver for QUBO
  """
  numpy_for_qubo solves the given QUBO using Numpy library functions
  """
  exact_mes = NumPyMinimumEigensolver()
  exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
  result = exact.solve(qubo)
  return result

In [17]:
def solve_QUBO(linear, quadratic, algo, p=1):
  """
  solve_QUBO is ahigher order function to solve QUBO using the given algo parameter function

  """
  keys = list(linear.keys())
  keys.sort(key=natural_keys)
  qubo = QuadraticProgram()
  for key in keys:
    qubo.binary_var(key)                                                             # initialize the binary variables
  qubo.minimize(linear=linear, quadratic=quadratic)                                  # initialize the QUBO maximization problem instance
  op, offset = qubo.to_ising()
  qp=QuadraticProgram()
  qp.from_ising(op, offset, linear=True)
  result = algo(qubo, p)
  return result

In [18]:
#convert solution binary string to coalition structure and also output the coalition
def decode(solution,coalition_values):
  """
  Convert the solution binary string into a coalition sructure(a list of coalitions)
  """
  output = []
  for index,element in enumerate(solution):
    if int(element)!=0:
      output.append(set(list(coalition_values)[index].split(',')))
  return output

In [19]:
def get_linear_quads(distribution,n):
  """
  wrapper function to fetch the linear and quadratic terms for a given distribution and number of agents
  """
  coalition_values = generate_problem_instance(distribution, n)
  c,S,b = convert_to_BILP(coalition_values)
  qubo_penalty =-50                                                 #lambda is a negative constant
  linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)
  return coalition_values,linear,quadratic

##Classical Solver (using Brute Force)

In [20]:
#Classical BILP solver

def constraint(x,S):
  """
  constraint(x,S) returns True if x is a valid binary string, else False
  """
  for i in range(len(S)):
    temp = 0
    for j in range(len(x)):
      temp+= (S[i][j]*int(x[j]))
    if temp!=1:
      return False
  return True


def solve_BILP_classical(coalition_values):
  """
  solve_BILP_classical() Evaluates the coalition value for all possible valid binary strings
  """
  S=[]
  for i in range(math.ceil(math.sqrt(len(coalition_values)))):                    #get number of agents
    S.append([])
    for j,value in coalition_values.items():
      if str(i+1) in j:
        S[i].append(1)
      else:
        S[i].append(0)
  optimal_cost = 0
  for b in range(2**len(coalition_values)):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(len(coalition_values))))]
    x = ''.join(str(e) for e in x)
    if constraint(x,S):
      cost = 0
      for j in range(len(x)):
        cost+=coalition_values[list(coalition_values.keys())[j]]*int(x[j])
      if optimal_cost<cost:
        optimal_cost = cost
        optimal_x = x
  return optimal_x

In [ ]:
table_headers = ['Distribution', 'No. of Agents','Solution Function Value','Solution Binary String',\
                 'QAOA Solution','True Solution (Classical)','QAOA Result','Time taken for QAOA',\
                 'Optimal p (QAOA layers)','Ranking Probability','Proability Distribution']
table_contents = []


distributions = [Agent_based_uniform,
               Agent_based_normal,
               Beta_distribution,
               Modified_normal_distribution,
               Modified_uniform_distribution,
               Normal_distribution,
               SVA_BETA_distribution,
               Weibull_distribution,
               Rayleigh_distribution,
               Weighted_random_with_chisquare,
               F_distribution,
               Laplace_or_double_exponential]
#distributions = [SVA_BETA_distribution]
n_agents = [2,3,4]
#n_agents = [3]

for distribution in distributions:
  for n in n_agents:
    print(f'Executing {distribution.__name__} distribution for {n} agents,',end=' ')
    
    coalition_values, linear, quadratic = get_linear_quads(distribution,n)


    p = 1                                            # number of QAOA layers
    classical_solution = 0
    qaoa_solution = 1


    classical_solution_binary_string = list(map(np.float,list(solve_BILP_classical(coalition_values))))
    classical_solution = decode(classical_solution_binary_string, coalition_values)

    while(classical_solution != qaoa_solution and p<=15):
      start_time = time.time()
      solution = solve_QUBO(linear,quadratic,qaoa_for_qubo,p)
      optimal_coalition_structure = decode(solution.x,coalition_values)
      qaoa_solution = decode(solution.x,coalition_values)
      tte = time.time()-start_time
      print(f'Time taken = {tte} for  p = {p}')
      #probabilities = []
      #for sample in solution.samples:
      #  if False not in (sample.x == classical_solution_binary_string):
      #    print('debug point--------------')
      #    true_solution_probability = sample.probability
      #  probabilities.append(sample.probability)
      #probabilities = sorted(probabilities, reverse=True)
      #print(f"probabilities={probabilities}")
      #print(f"true_solution_probability={true_solution_probability}")
      #for  idx,prob in enumerate(probabilities):
      #  if (prob==true_solution_probability):
      #    rank = idx
      #    break
      #[idx for idx,prob in enumerate(probabilities) if (prob==true_solution_probability)]
      #rank = probabilities.index(true_solution_probability)
      #print("List=====",[sample.probability for sample in solution.samples])
      #print("True solution probability=====",[bin_str.probability for bin_str in solution.samples if False not in (bin_str.x == classical_solution_binary_string)])
      #print("total samples =",len(solution.samples))
      try:
        rank = 1 + sorted([sample.probability for sample in solution.samples],reverse=True).index([bin_str.probability for bin_str in solution.samples if False not in (bin_str.x == classical_solution_binary_string)][0])
      except:
        print("Exception Handled!!!!!!!!!! Trying again.....")
        continue
      p+=1

    row = []
    row.append(distribution.__name__)                              #distribution used
    row.append(n)                                                  #number of agents
    row.append(solution.fval)                                      #Solution Function Value
    row.append(list(map(int,solution.x)))                          #Solution Binary String
    row.append(qaoa_solution)                                      #QAOA Solution
    row.append(classical_solution)                                 #True solution
    row.append(classical_solution == qaoa_solution)                #Correctness of the Quantum solution
    row.append(tte)                                                #Time to execute the QAOA circuit
    row.append(p if classical_solution == qaoa_solution else None) #value of p, number of layers, if the quantum solution is correct
    row.append(f"{rank}/{len(solution.samples)}")                  #Probability rank of the True solution
    row.append(solution.samples)                                   #Proability distribution generated
    table_contents.append(row)
    print('\n')

##Execute Functions and Generate

In [36]:
#view output table
def highlight_false(s, column):
    is_false = pd.Series(data=False, index=s.index)
    is_false[column] = s.loc[column] == False
    return ['background-color: black' if is_false.any() else '' for v in is_false]

df = pd.DataFrame(table_contents, columns=table_headers)
#df.to_pickle('/content/drive/MyDrive/'+'report.pkl')

df.drop('Proability Distribution', axis=1).style.apply(highlight_false, column='QAOA Result', axis=1)

,Distribution,No. of Agents,Solution Function Value,Solution Binary String,QAOA Solution,True Solution (Classical),QAOA Result,Time taken for QAOA,Optimal p (QAOA layers),Ranking Probability
0,SVA_BETA_distribution,2,-450.333000,"[1, 0, 1]","[{'1'}, {'1', '2'}]","[{'1'}, {'2'}]",False,2.442580,None,3/8
1,SVA_BETA_distribution,3,-552.919000,"[1, 0, 1, 0, 1, 0, 0]","[{'1'}, {'1', '2'}, {'1', '3'}]","[{'2'}, {'1', '3'}]",False,18.283749,None,62/119


In [ ]:
#Classical BILP solver     2,3,4,5(takes a lot of time)

#Classical QUBO solver     2,3,4,5(resource overflow)gith

#QAOA QUBO solver          2,3,4(15 qubits)(resource overflow),5(31 qubits)(resource overflow)